In [3]:
import argparse
import sys
import os
import time
import csv
import cv2
import mediapipe as mp
import numpy as np
from pathlib import Path

from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2

In [4]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [5]:
model = "mediapipe_pose_landmarker_model/pose_landmarker_heavy.task"

base_options = python.BaseOptions(model_asset_path=model)
options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        running_mode=vision.RunningMode.VIDEO,
        num_poses=1,
        min_pose_detection_confidence=0.5,
        min_pose_presence_confidence=0.5,
        min_tracking_confidence=0.5,
        output_segmentation_masks=False)

detector = vision.PoseLandmarker.create_from_options(options)

In [6]:
def get_total_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return total_frames

In [7]:
def prepare_csv_file(filename):
    csvfile_handler = open(filename, 'w', newline='')
    landmark_writer = csv.writer(csvfile_handler, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    headers = []
    headers.extend([f'videoid'])
    headers.extend([f'framecount'])
    for i in range(33):  # For each of the 33 landmarks
        headers.extend([f'Landmark_{i}_X', f'Landmark_{i}_Y', f'Landmark_{i}_Z', f'Landmark_{i}_Visibility'])
    landmark_writer.writerow(headers)  
    
    return csvfile_handler, landmark_writer

In [8]:
def build_landmark_row(videoid, frame_count, pose_landmarks):
    row = []
    row.extend([videoid])
    row.extend([frame_count])
    
    def add_lanmark(index):
        landmark = pose_landmarks[0][index]
        row.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])

    for i in range(33):
        add_lanmark(i)
    '''
    landmark_ids = [0, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
    
    for landmark_id in landmark_ids:
        add_lanmark(landmark_id)
    '''
    return row

In [9]:
def draw_landmarks(frame, res_pose_landmarks):
    for pose_landmarks in res_pose_landmarks:
     # Draw the pose landmarks.
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y,
                z=landmark.z) for landmark
                in pose_landmarks
            ])
        mp_drawing.draw_landmarks(
            frame,
            pose_landmarks_proto,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing_styles.get_default_pose_landmarks_style())
    return frame

In [10]:
def process_frame(frame, video_width, video_height, bFlip, percentCrop, rotateAngle, bNoise, bBlur):
    if bFlip:
        frame = cv2.flip(frame, 1)

    if percentCrop != 0:
        original_width = video_width
        original_height = video_height
        crop_width = int(original_width * (1 - percentCrop/100))
        crop_height = int(original_height * (1 - percentCrop/100))
        x_start = (original_width - crop_width) // 2
        y_start = (original_height - crop_height) // 2

        frame = frame[y_start:y_start + crop_height, x_start:x_start + crop_width]
        frame = cv2.resize(frame, (original_width, original_height))  # Resize back to original dimensions


    if rotateAngle != 0:
        angle = rotateAngle
        (h, w) = frame.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        frame = cv2.warpAffine(frame, M, (w, h))
        
    if bNoise:
        noise = np.zeros(frame.shape, frame.dtype)
        cv2.randn(noise, 0, 25)
        frame = cv2.add(frame, noise)

    if bBlur:
        frame = cv2.GaussianBlur(frame, (3, 3), 0) 
        
    return frame

In [11]:
debug_allowed = True

def process_video(row):    
    global landmark_fall_writer, landmark_notfall_writer, video_processed_count
    
    video_filename = row['Filename']
    video_path = "dataset/" + video_filename

    bFlip = int(row['Flip']) 
    percentCrop = int(row['Crop'])
    rotateAngle = int(row['Rotate'])
    bNoise = int(row['Noise'])
    bBlur = int(row['Blur'])
    if bFlip:
        video_filename = video_filename + "_flip"
    if percentCrop != 0:
        video_filename = video_filename + "_crop" + str(percentCrop)
    if rotateAngle != 0:
       video_filename = video_filename + "_rotate" + str(rotateAngle)
    if bNoise:
        video_filename = video_filename + "_noise"
    if bBlur:
        video_filename = video_filename + "_blur"
        
    video_filename = video_filename + "_" + str(video_processed_count)
    
    video_begin_frame = int(row['VideoBeginFrame'])
    video_end_frame = int(row['VideoEndFrame']) if int(row['VideoEndFrame']) != -1 else get_total_frames(video_path)
    fall_begin_frame = int(row['FallBeginFrame'])
    fall_end_frame = int(row['FallEndFrame'])
    print("Reading from video file ", video_path)
    video = cv2.VideoCapture(video_path)
    
    detector = vision.PoseLandmarker.create_from_options(options)
    
    good_dataframe = 0
    
    fps = video.get(cv2.CAP_PROP_FPS)
    if fps < 29 or fps > 33:
        print("Invalid video: FPS != 30 - ", fps)
        return

    if video_begin_frame == -1:
        findex = fall_begin_frame - 120
        if findex < 0:
            findex = 0
            
        video.set(cv2.CAP_PROP_POS_FRAMES, findex)
        frame_count = findex
    else:
        real_vid_begin_frame = video_begin_frame - 120
        if real_vid_begin_frame < 0:
            real_vid_begin_frame = 0
        
        video.set(cv2.CAP_PROP_POS_FRAMES, real_vid_begin_frame)
        frame_count = real_vid_begin_frame
        

    last_pos_msec = 0

    notfall_blockA_framecount = 0
    notfall_blockB_framecount = 0
    fall_block_framecount = 0

    drop_frame_state = 0


    while True:
        success, frame = video.read()
        if not success:
            break  
            
        #frame = cv2.convertScaleAbs(frame, beta=100)
    
        video_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))   
        frame = process_frame(frame, video_width, video_height, bFlip, percentCrop, rotateAngle, bNoise, bBlur)
        
            
        rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)

        if last_pos_msec > video.get(cv2.CAP_PROP_POS_MSEC):
            break
            
        last_pos_msec = int(video.get(cv2.CAP_PROP_POS_MSEC))
        results = detector.detect_for_video(mp_image, last_pos_msec)
        if results.pose_landmarks:
            if drop_frame_state == 2:
                pass
                #print("Dropped frame detected. Exiting loop")
                #break
                
            drop_frame_state = 1
            
            good_dataframe += 1
            collecting_data = False
            
            if fall_begin_frame != -1 and fall_end_frame != -1:
             	if frame_count in range(fall_begin_frame, fall_end_frame):
                    row = build_landmark_row(video_filename, frame_count, results.pose_landmarks)
                    landmark_fall_writer.writerow(row)
                    fall_block_framecount += 1
                    collecting_data = True
            if video_begin_frame != -1:
                end_zone = fall_begin_frame - 1
                if fall_begin_frame == -1:
                    end_zone = video_end_frame
                if frame_count in range(video_begin_frame, end_zone):
                    row = build_landmark_row(video_filename + "_b1", frame_count, results.pose_landmarks)
                    landmark_notfall_writer.writerow(row)
                    notfall_blockA_framecount += 1
                    collecting_data = True
                if fall_end_frame != -1:
                    if frame_count in range(fall_end_frame+1, video_end_frame):
                        row = build_landmark_row(video_filename + "_b2", frame_count, results.pose_landmarks)
                        landmark_notfall_writer.writerow(row)          
                        notfall_blockB_framecount += 1
                        collecting_data = True
            if debug_allowed and collecting_data:
                frame = draw_landmarks(frame, results.pose_landmarks)
        else:
            if drop_frame_state == 1:
                drop_frame_state = 2
            
        frame_count += 1      
        if (video_begin_frame == -1 and frame_count >= fall_end_frame) or (frame_count >= video_end_frame):
            break

        if debug_allowed:      
            cv2.imshow("Image", frame)
            if cv2.waitKey(1) == ord('q'):
               break
            
    #if good_dataframe > 0:
    #    print("good_dataframe:", good_dataframe)
        
    print("video =", video_filename, "fall_block_framecount =", fall_block_framecount, ", notfall_blockA_framecount =", notfall_blockA_framecount, "notfall_blockB_framecount =", notfall_blockB_framecount)
    video.release()

In [11]:
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed

video_processed_count = 0
# Assuming your prepare_csv_file and process_video functions are defined elsewhere
'''
fall_csvfile, landmark_fall_writer = prepare_csv_file('dataset/may/train/FALL.csv')
notfall_csvfile, landmark_notfall_writer = prepare_csv_file('dataset/may/train/NOT_FALL.csv')
csv_label_file_path = "dataset/may/train/augmented_data.csv"
'''
'''
fall_csvfile, landmark_fall_writer = prepare_csv_file('dataset/may/val/FALL.csv')
notfall_csvfile, landmark_notfall_writer = prepare_csv_file('dataset/may/val/NOT_FALL.csv')
csv_label_file_path = "dataset/may/val/augmented_data.csv"
'''
'''
fall_csvfile, landmark_fall_writer = prepare_csv_file('dataset/may/test/FALL.csv')
notfall_csvfile, landmark_notfall_writer = prepare_csv_file('dataset/may/test/NOT_FALL.csv')
csv_label_file_path = "dataset/may/test/augmented_data.csv"
'''
fall_csvfile, landmark_fall_writer = prepare_csv_file('dataset/pi_test_set/FALL.csv')
notfall_csvfile, landmark_notfall_writer = prepare_csv_file('dataset/pi_test_set/NOT_FALL.csv')
csv_label_file_path = "dataset/pi_test_set/data2.csv"

# Function wrapper to handle both regular and flipped processing
def process_video_wrapper(row):
    global video_processed_count
    video_processed_count += 3;
    process_video(row)


# Load all rows into memory; ensure your dataset can fit into RAM
all_rows = []
with open(csv_label_file_path, mode='r', newline='') as file:
    csv_dataset_label = csv.DictReader(file)
    all_rows = [row for row in csv_dataset_label]

# Use ThreadPoolExecutor to process videos in parallel
with ThreadPoolExecutor(max_workers=40) as executor:
    # Create future tasks for both regular and flipped processing for each video
    futures = [executor.submit(process_video_wrapper, row) for row in all_rows]
    
    # Wait for all futures to complete, you can also handle results or exceptions here
    for future in as_completed(futures):
        try:
            result = future.result()
            # Optionally handle result or increment counter
            video_processed_count += 1
        except Exception as e:
            print(f"An error occurred: {e}")

fall_csvfile.close()
notfall_csvfile.close()

Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
video = videos\Le2i\Office\video (11).out.avi_noise_15 fall_block_framecount = 0 , notfall_blockA_framecount = 0 notfall_blockB_framecount = 0
video = videos\Le2i\Office\video (11).out.avi_3 fall_block_framecount = 38 , notfall_blockA_framecount = 0 notfall_blockB_framecount = 0
video = videos\Le2i\Office\video (11).out.avi_blur_18 fall_block_framecount = 46 , notfall_blockA_framecount = 0 notfall_blockB_framecount = 0
video = videos\Le2i\Office\video (11).out.avi_flip_6 fall_block_framecount = 47 , notfall_blockA_framecount = 0 notfall_blockB_framecount = 0
video = v

In [16]:

fall_csvfile, landmark_fall_writer = prepare_csv_file('dataset/pi_test_set/FALL.csv')
notfall_csvfile, landmark_notfall_writer = prepare_csv_file('dataset/pi_test_set/NOT_FALL.csv')
csv_label_file_path = "dataset/pi_test_set/data2.csv"

video_processed_count = 0
with open(csv_label_file_path, mode='r', newline='') as file:
    csv_dataset_label = csv.DictReader(file)
    for row in csv_dataset_label:
        process_video(row)
    
fall_csvfile.close()
notfall_csvfile.close()

Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
video = videos\Le2i\Office\video (11).out.avi_0 fall_block_framecount = 38 , notfall_blockA_framecount = 0 notfall_blockB_framecount = 0
Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
video = videos\Le2i\Office\video (11).out.avi_flip_0 fall_block_framecount = 47 , notfall_blockA_framecount = 0 notfall_blockB_framecount = 0
Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
video = videos\Le2i\Office\video (11).out.avi_crop10_0 fall_block_framecount = 47 , notfall_blockA_framecount = 0 notfall_blockB_framecount = 0
Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
video = videos\Le2i\Office\video (11).out.avi_rotate10_0 fall_block_framecount = 47 , notfall_blockA_framecount = 0 notfall_blockB_framecount = 0
Reading from video file  dataset/videos\Le2i\Office\video (11).out.avi
video = videos\Le2i\Office\video (11).out.avi_noise_0 fall_block_framecount 

In [29]:
fall_csvfile.close()
notfall_csvfile.close()